In [176]:
using LazySets
using LazySets: dot
function σ(d::AbstractVector{N}, P::VPolygon{N}) where {N <: Real}
    function up(u, v)
        return (dot(u, v) > 0)
    end
    function down(u, v)
        return (dot(u, v) < 0)
    end
    function dr(u, Vi, Vj)
        return (dot(u, (Vi) - (Vj))) # direction sign of (Vi-Vj)
    end
    function above(u, Vi, Vj)
        return (dr(u, Vi, Vj) > 0) # true if Vi is above Vj
    end
    function below(u, Vi, Vj)
        return (dr(u, Vi, Vj) < 0) # true if Vi is below Vj
    end
    
    push!(P.vertices, P.vertices[1]) #add extra vertice on the end equal to the first
    
    n = length(P.vertices)
    a = 1; b = n # start chain = [1,n] with P.vertices[n]=P.vertices[0]
    A = P.vertices[2] - P.vertices[1]
    upA = up(d, A)
    # test if P.vertices[0] is a local maximum
    if (!upA && !above(d, P.vertices[n - 1], P.vertices[1])) # P.vertices[0] is the maximum
        return P.vertices[1]
    end
    while true
        c = round(Int, (a + b) / 2) # midpoint of [a,b], and 0<c<n
        C = P.vertices[c + 1] - P.vertices[c]
        upC = up(d, C)
        if (!upC && !above(d, P.vertices[c - 1], P.vertices[c])) # P.vertices[c] is a local maximum
            pop!(P.vertices) # remove the extra point added
            return P.vertices[c] # thus it is the maximum
        end
        # no max yet, so continue with the  binary search
        # pick one of the two subchains [a,c]  or [c,b]
        if upA # A points up
            if !upC # C points down
                b = c
            else # C points up
                if above(d, P.vertices[a], P.vertices[c]) # P.vertices[a] above P.vertices[c]
                    b = c
                else # P.vertices[a] below P.vertices[c]
                    a = c
                    A = C
                    upA = upC
                end
            end
            else # A points down
            if upC # C points up
                a = c
                A = C
                upA = upC
            else # C points down
                if below(d, P.vertices[a], P.vertices[c]) # P.vertices[a] below P.vertices[c]
                    b = c
                else # P.vertices[a] above P.vertices[c]
                    a = c
                    A = C
                    upA = upC
                end
            end
        end
        if (b <= a+1) # the chain is impossibly small
            pop!(P.vertices) # remove the extra point added
            throw(ErrorException("something went wrong")) # return an error
        end
    end
end

σ (generic function with 1 method)

In [181]:
using BenchmarkTools
d = [1.0, 1.0]
P = rand(VPolygon, num_vertices=10000)
@btime LazySets.σ($d, P)

  761.368 μs (9999 allocations: 937.41 KiB)


2-element Array{Float64,1}:
 3.67620537160116  
 6.1676861479660845

In [182]:
@btime σ($d, $P)

  2.230 μs (27 allocations: 2.53 KiB)


2-element Array{Float64,1}:
 3.67620537160116  
 6.1676861479660845